<h2>Essential Libraries</h2>

In [5]:
import os
import pygame
from PIL import Image
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List
from collections import defaultdict

In [6]:
print(os.getcwd())
root = "../src/assets/map/"

d:\Coding\War of the ring\War-of-the-Ring\notebook


<h2>Classes</h2>

<h2>Utility Functions</h2>

<h2>Game Variables</h2>

In [7]:
# pygame window dimensions
WINDOW_WIDTH = 1920
WINDOW_HEIGHT = 1080

In [8]:
# map dimension
map_image = Image.open(root + "map_cut.png")
map_mask = Image.open(root + "map_mask.png")

MAP_WIDTH = map_image.width
MAP_HEIGHT = map_image.height

In [9]:
print(f"map width = {MAP_WIDTH}")
print(f"map height = {MAP_HEIGHT}")

map width = 6854
map height = 4708


In [10]:
mapheight_fit_factor = WINDOW_HEIGHT/MAP_HEIGHT*0.75

In [11]:
game_area = {
    "x0" : 100, "y0" : 0,
    "width" : mapheight_fit_factor*MAP_WIDTH, "height" : mapheight_fit_factor*MAP_HEIGHT 
}

In [12]:
class TextClass:
    def __init__(self, text_content : str , font_loc : str,
                 font_size : int, font_color : Tuple[int, int, int],
                 rect_center : Tuple[int, int]):
        self.text_content = text_content
        self.font_loc = font_loc
        self.font_size = font_size
        self.font_color = font_color
        self.font = pygame.font.Font(self.font_loc, self.font_size)
        self.text = self.font.render(self.text_content, True, self.font_color)
        self.text_rect = self.text.get_rect()
        self.text_rect.center = rect_center

    def draw_text(self, screen : pygame.Surface):
        screen.blit(self.text, self.text_rect)

class Quit_Box:
    def __init__(self, text_values : dict, 
                 x : int = 0, y : int = 0,
                 width : int = 100, height : int = 50,
                 ):
        # box
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        # text
        text_values['rect_center'] = (self.x + self.width/2, self.y + self.height/2)
        self.text = TextClass(**text_values)

    def draw(self, screen : pygame.Surface) -> None:
        # draw quit game box at top left
        pygame.draw.rect(screen, (255, 255, 255), pygame.Rect(self.x, self.y, self.width, self.height))
        self.text.draw_text(screen)

    def is_inside(self, mouse_x : int, mouse_y : int) -> bool:
        if mouse_x >= self.x and mouse_x <= self.x + self.width\
        and mouse_y >= self.y and mouse_y <= self.y + self.height:
            return True
        return False

In [13]:
def get_map_coordinate(mouse_x : int, mouse_y : int, x_offset, y_offset, zoom) -> Tuple[int, int]:
    map_x = (mouse_x + x_offset - game_area['x0']) // (mapheight_fit_factor * zoom) 
    map_y = (mouse_y + y_offset - game_area['y0']) // (mapheight_fit_factor * zoom)
    return int(map_x), int(map_y)

In [14]:
def rgb2hex(r : int, g : int, b : int) -> str:
    return "{:02x}{:02x}{:02x}".format(r, g, b)

In [15]:
def get_region_name(region_color : str, df : pd.DataFrame) -> str:
    return df[df['Color-Hex'] == region_color]['Name'].iloc[0]

In [16]:
df = pd.read_csv(root + 'map_mask_data.csv')
df.head()

,Name,Type,Color-Decimal,Color-Hex
0,Anfalas,region,100,000064
1,Angmar,region,200,0000C8
2,Arnor,region,400,000190
3,Ash Mountains,region,500,0001F4
4,Barad Dur,region,600,000258


In [17]:
zoom = 1
zoom_step = 0.3
zoom_min = 1
zoom_max = 5

def get_scaled_map(map_image : pygame.Surface) -> pygame.Surface:
    return pygame.transform.smoothscale_by(map_image, mapheight_fit_factor * zoom)

In [18]:
def bound_game_area(x0, y0, map_width, map_height):
    x0 = min(x0, map_width - game_area['width'])
    x0 = max(x0, 0)
    y0 = min(y0, map_height - game_area['height'])
    y0 = max(y0, 0)
    return x0, y0

In [19]:
def get_view_rect(map_width : int, map_height : int, offset_x, offset_y, zoom_type) -> Tuple[int, int]:
    mouse_x, mouse_y = pygame.mouse.get_pos()

    camera_center = (offset_x + game_area['width']/2, offset_y + game_area['height']/2)
    new_camera_center = (mouse_x + offset_x, mouse_y + offset_y)
    camera_drift_direction =  (new_camera_center[0] - camera_center[0], new_camera_center[1] - camera_center[1])
    
    x0 = offset_x + camera_drift_direction[0]*0.5*zoom_type
    y0 = offset_y + camera_drift_direction[1]*0.5*zoom_type
    x0, y0 = bound_game_area(x0, y0, map_width, map_height)

    return (x0, y0)

In [20]:
def inside_game_area():
    mouse_x, mouse_y = pygame.mouse.get_pos()
    
    return mouse_x >= game_area['x0'] and mouse_x <= game_area['x0'] + game_area['width']\
    and mouse_y >= game_area['y0'] and mouse_y <= game_area['y0'] + game_area['height']

<h2>Main Loop</h2>

In [22]:
region_masks = {}  # color_hex -> boolean mask
map_mask_arr = np.array(map_mask)

# Create 3-channel RGB version
region_color_map = {}  # color tuple -> hex string
map_rgb = map_mask_arr[:, :, :3]
h, w = map_rgb.shape[:2]
unique_colors = np.unique(map_rgb.reshape(-1, 3), axis=0)

for color in unique_colors:
    print(color)
    color_tuple = tuple(color.tolist())
    color_hex = rgb2hex(*color_tuple).upper()
    region_color_map[color_tuple] = color_hex
    mask = np.all(map_rgb == color_tuple, axis=-1)
    region_masks[color_hex] = mask

[0 0 0]
[  0   0 100]
[  0   0 200]
[  0   1 144]
[  0   1 244]
[ 0  2 88]
[  0   2 188]
[ 0  3 32]
[  0   3 132]
[  0   3 232]
[ 0  4 76]
[  0   4 176]
[ 0  5 20]
[  0   5 120]
[  0   5 220]
[ 0  6 64]
[  0   6 164]
[0 7 8]
[  0   7 108]
[  0   7 208]
[ 0  8 52]
[  0   8 152]
[  0   8 252]
[ 0  9 96]
[  0   9 196]
[ 0 10 40]
[  0  10 140]
[  0  10 240]
[ 0 11 84]
[  0  11 184]
[ 0 12 28]
[  0  12 128]
[  0  12 228]
[ 0 13 72]
[  0  13 172]
[ 0 14 16]
[  0  14 116]
[  0  14 216]
[ 0 15 60]
[  0  15 160]
[ 0 16  4]
[  0  16 104]
[  0  16 204]
[ 0 17 48]
[  0  17 148]
[  0  17 248]
[ 0 18 92]
[  0  18 192]
[ 0 19 36]
[  0  19 136]
[  0  19 236]
[ 0 20 80]
[  0  20 180]
[ 0 21 24]
[  0  21 124]
[  0  21 224]
[ 0 22 68]
[  0  22 168]
[ 0 23 12]
[  0  23 112]
[  0  23 212]
[ 0 24 56]
[  0  24 156]
[ 0 25  0]
[  0  25 100]
[  0  25 200]
[ 0 26 44]
[  0  26 144]
[  0  26 244]
[ 0 27 88]
[  0  27 188]
[ 0 28 32]
[  0  28 132]
[  0  28 232]
[ 0 29 76]
[  0  29 176]
[ 0 30 20]
[  0  30 120]
[  0

In [26]:
region_masks

{'000000': array([[ True,  True, False, ..., False,  True,  True],
        [ True,  True, False, ..., False,  True,  True],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [ True,  True, False, ..., False,  True,  True],
        [ True,  True, False, ..., False,  True,  True]],
       shape=(4708, 6854)),
 '000064': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],
       shape=(4708, 6854)),
 '0000C8': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, Fa

In [28]:
zoom = 1
zoom_type = 1

# Initialize Pygame
pygame.init()
# ui elements
text_values = {
    "text_content" : "Quit",
    "font_loc" : root + "font.ttf",
    "font_size" : 30, "font_color" : (0, 0, 0),
    "rect_center" : (0, 0)
}

quit_box = Quit_Box(text_values=text_values)
pygame.display.set_caption("War of the Ring")
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT), pygame.FULLSCREEN, 32)
original_map_image = pygame.image.load(root + "map_cut.png").convert()
map_image = get_scaled_map(original_map_image)
view_rect = pygame.Rect(0, 0, game_area['width'], game_area['height'])
clock = pygame.time.Clock()
running = True
dragging = False
offset_x = 0
offset_y = 0
image_start = (offset_x, offset_y)

map_mask_overlay = np.zeros_like(map_mask_arr)
map_mask_surface = pygame.image.frombuffer(map_mask_overlay.tobytes(),
                            (map_mask_overlay.shape[0], map_mask_overlay.shape[1]), 'RGBA')
map_mask_surface_scaled = get_scaled_map(map_mask_surface)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        
        # mouse button down
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # left click
            if event.button == 1:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                # quit button
                if quit_box.is_inside(mouse_x, mouse_y):
                    running = False
                # button click on game area
                elif inside_game_area():
                    print("Inside game area")
                    map_x, map_y = get_map_coordinate(mouse_x, mouse_y, offset_x, offset_y, zoom)
                    region_color = map_mask.getpixel((map_x, map_y))[:3]
                    region_color_hex = rgb2hex(*region_color).upper()

                    # Fast copy from cached mask
                    map_mask_overlay[:] = 0  # clear previous
                    if region_color_hex in region_masks:
                        mask = region_masks[region_color_hex]
                        map_mask_overlay[mask] = [255, 0, 0, 100]  # RGBA highlight

                    # Recreate surface
                    map_mask_surface = pygame.image.frombuffer(
                        map_mask_overlay.tobytes(),
                        (map_mask_overlay.shape[1], map_mask_overlay.shape[0]),
                        'RGBA'
                    )
                    map_mask_surface_scaled = get_scaled_map(map_mask_surface)
                    
                    print(f"Region Color : {region_color_hex}")
                    region_name = get_region_name(region_color_hex.upper(), df)
                    print(f"Region name : {region_name}")
                # button click outside game area
                else:
                    print("Outside game area")
            
            # right click to drag
            elif event.button == 3:
                dragging = True
                drag_start = pygame.mouse.get_pos()
                image_start = (offset_x, offset_y)
            
            # scroll up
            elif (event.button == 4 or event.button == 5) and inside_game_area():
                if event.button == 4:
                    zoom = min(zoom_max, zoom + zoom_step)
                    zoom_type = 1
                else:
                    zoom = max(zoom_min, zoom - zoom_step)
                    zoom_type = -1
                map_image = get_scaled_map(original_map_image)
                map_mask_surface_scaled = get_scaled_map(map_mask_surface)
                offset_x, offset_y = get_view_rect(map_image.get_width(), map_image.get_height(), offset_x, offset_y, zoom_type)
                view_rect = pygame.Rect(offset_x, offset_y, game_area['width'], game_area['height'])
        
        # end of dragging
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 3:  # Stop dragging
                dragging = False

        elif event.type == pygame.MOUSEMOTION and dragging:
            mouse_x, mouse_y = pygame.mouse.get_pos()
            dx = mouse_x - drag_start[0]
            dy = mouse_y - drag_start[1]
            offset_x = image_start[0] - dx
            offset_y = image_start[1] - dy
            offset_x, offset_y = bound_game_area(offset_x, offset_y, map_image.get_width(), map_image.get_height())
            view_rect = pygame.Rect(offset_x, offset_y, game_area['width'], game_area['height'])

    screen.fill((0, 0, 0))

    # render the frame
    # map
    screen.blit(map_image, (100, 0), view_rect)
    screen.blit(map_mask_surface_scaled, (100, 0), view_rect)  # Add this line to draw highlight
    # ui
    quit_box.draw(screen)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

Inside game area
Region Color : 002328
Region name : Tharbad
Inside game area
Region Color : 000A8C
Region name : Enedwaith
Inside game area
Region Color : 001518
Region name : Minhiriath
Inside game area
Region Color : 000384
Region name : Cardolan
Inside game area
Region Color : 0020D0
Region name : South Ered Luin
Inside game area
Region Color : 0023F0
Region name : Tower Hills
Inside game area
Region Color : 000384
Region name : Cardolan
Inside game area
Region Color : 000320
Region name : Buckland
Inside game area
Region Color : 0002BC
Region name : Bree
Inside game area
Region Color : 0025E4
Region name : Weather Hills
Inside game area
Region Color : 002454
Region name : Trollshaws
Inside game area
Region Color : 001F40
Region name : Rivendell
Inside game area
Region Color : 001F40
Region name : Rivendell
Inside game area
Region Color : 000DAC
Region name : Fords of Bruinen
Inside game area
Region Color : 001F40
Region name : Rivendell
Inside game area
Region Color : 001194
Regio

<h2>Testing</h2>

In [38]:
arr = np.array(map_mask)
indices = np.argwhere(np.all(arr == [0, 0, 0, 255], axis=-1))

In [41]:
new_array = np.zeros_like(arr)

In [43]:
new_array[indices[:, 0], indices[:, 1]] = [255, 0, 0, 100]

In [47]:
new_array.shape

(4708, 6854, 4)

In [48]:
pygame.image.frombuffer(new_array.tobytes(), (4708, 6854), 'RGBA')

<Surface(4708x6854x32 SW)>

In [46]:
new_array.shape

(4708, 6854, 4)

In [ ]:
small_array = array[:50, :50, :]  # Extract a (50, 50, 4) array
surface = pygame.surfarray.make_surface(small_array)